In [82]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import warnings
import math
from tqdm import tqdm
from irsx.xmlrunner import XMLRunner
xml_runner = XMLRunner()
warnings.filterwarnings('ignore')


In [83]:
file=pd.read_csv('union_ein_name_list1.csv')
file_EIN=list(file['EIN'])
df_index=pd.DataFrame()
for year in range(2015, 2020):
    try:
        print(str(year), 'started', end='\t')
        df_index_temp=pd.read_csv('https://s3.amazonaws.com/irs-form-990/index_'+str(year)+'.csv', 
                                  error_bad_lines=False, warn_bad_lines=True # Be cautious with bad lines.
                                 ).query('EIN in @file_EIN')
        df_index_temp['YEAR']=year
        df_index=pd.concat([df_index, df_index_temp], ignore_index=True)
        print('got')
    except:
        print('passed')


2015 started	got
2016 started	got
2017 started	got
2018 started	got
2019 started	got


In [84]:
df_index.groupby(['YEAR','RETURN_TYPE']).count()

RETURN_ID  FILING_TYPE  EIN  TAX_PERIOD  SUB_DATE  \
YEAR RETURN_TYPE                                                      
2015 990                119          119  119         119       119   
     990EO               30           30   30          30        30   
     990EZ               95           95   95          95        95   
     990O                33           33   33          33        33   
     990PF               72           72   72          72        72   
2016 990                213          213  213         213       213   
     990EO               38           38   38          38        38   
     990EZ              148          148  148         148       148   
     990O                52           52   52          52        52   
     990PF               93           93   93          93        93   
2017 990                252          252  252         252       252   
     990EO               51           51   51          51        51   
     990EZ              154          154  154         154       154   
     990O                83           83   83          83        83   
     990PF              117          117  117         117       117   
2018 990                245          245  245         245       245   
     990EO               42           42   42          42        42   
     990EZ              121          121  121         121       121   
     990O                71           71   71          71        71   
     990PF              108          108  108         108       108   
2019 990                136          136  136         136       136   
     990EO               16           16   16          16        16   
     990EZ               67           67   67          67        67   
     990O                47           47   47          47        47   
     990PF               65           65   65          65        65   

                  TAXPAYER_NAME  DLN  OBJECT_ID  
YEAR RETURN_TYPE                                 
2015 990                    119  119        119  
     990EO                   30   30         30  
     990EZ                   95   95         95  
     990O                    33   33         33  
     990PF                   72   72         72  
2016 990                    213  213        213  
     990EO                   38   38         38  
     990EZ                  148  148        148  
     990O                    52   52         52  
     990PF                   93   93         93  
2017 990                    252  252        252  
     990EO                   51   51         51  
     990EZ                  154  154        154  
     990O                    83   83         83  
     990PF                  117  117        117  
2018 990                    245  245        245  
     990EO                   42   42         42  
     990EZ                  121  121        121  
     990O                    71   71         71  
     990PF                  108  108        108  
2019 990                    136  136        136  
     990EO                   16   16         16  
     990EZ                   67   67         67  
     990O                    47   47         47  
     990PF                   65   65         65

In [85]:
df_index.sample(10)

,RETURN_ID,FILING_TYPE,EIN,TAX_PERIOD,SUB_DATE,TAXPAYER_NAME,RETURN_TYPE,DLN,OBJECT_ID,YEAR
605,13730349,EFILE,956926706,201512,08/23/2016,TUA WILLIAM E SCHNEIDER CHAR,990PF,93491098007836,201630989349100783,2016
1188,14665306,EFILE,916310582,201612,8/22/2017 12:26:54 PM,EVA AND ARRA WOODS CHARITABLE TRUST,990PF,93491097008077,201720979349100807,2017
515,13190756,EFILE,264702901,201412,02/09/2016,VETERANSPLUS,990,93493320103925,201523209349310392,2016
1967,15838408,EFILE,742590850,201709,10/26/2018 4:41:50 PM,KEEP HARLINGEN BEAUTIFUL INC,990EZ,93492222003388,201832229349200338,2018
1652,15154659,EFILE,760691272,201708,1/30/2018 2:37:47 PM,GOLFREACH INC,990EZ,93492287002187,201732879349200218,2018
1873,15648137,EFILE,746355245,201712,9/4/2018 7:24:35 AM,FLOY M CADWALLADER RELIGIOUS TRUST,990PF,93491116000078,201821169349100007,2018
637,13734839,EFILE,741498701,201512,08/24/2016,RIVERSIDE WATER SUPPLY CORPORATION,990O,93493112002126,201621129349300212,2016
572,13669418,EFILE,611603933,201509,08/01/2016,NATIONAL COUNCIL OF THE UNITED STATES SOCIETY ...,990EZ,93492132021666,201611329349202166,2016
1401,14938520,EFILE,752708139,201612,11/8/2017 4:33:12 PM,FRIENDS OF THE KATY TRAIL INC,990,93493269015157,201702699349301515,2017
1725,15203627,EFILE,752254312,201709,3/6/2018 8:30:53 AM,KEEP MESQUITE BEAUTIFUL INC,990EZ,93492039002088,201830399349200208,2018


In [86]:
def func_text_acq(index):
    global df_index, done_index_list, df_index_text_acq, counter
    
    result_990, result_990EZ, result_990PF=[], [], []
    def form_990(object_id, name, year, EID):
        IRS990_result = xml_runner.run_sked(object_id, 'IRS990').result

        if IRS990_result!=None:
            IRS990_result=IRS990_result[0]
        else:
            re = [[object_id], [EID], [name], [year], ['No Result']]
            for i in range(12):
                re.append([0])
            return re


        #判断是Trust，Corporation还是什么
        if 'OfOrgnztnTrstInd' in IRS990_result['schedule_parts']['part_0']:
            var_0=['Trust']
        elif'OthrOrgnztnDsc' in IRS990_result['schedule_parts']['part_0']:
            var_0=['Other Description']
        elif 'OfOrgnztnCrpInd' in IRS990_result['schedule_parts']['part_0']:
            var_0=['Corporation']
        elif 'OfOrgnztnOthrInd' in IRS990_result['schedule_parts']['part_0']:
            var_0=['Other']
        elif 'OfOrgnztnAsscInd' in IRS990_result['schedule_parts']['part_0']:
            var_0=['Association']
        else:
            var_0=['No Result']
        #part 1 继续加变量
        variable_1 ='TtlCntrbtnsAmt' 
        variable_2='TtlPrgrmSrvcRvnAmt'
        variable_3='CYInvstmntIncmAmt'
        variable_4='CYOthrRvnAmt'
        variable_5='CYTtlRvnAmt'
        variable_6='CYGrntsAndSmlrPdAmt'
        variable_7='CYBnftsPdTMmbrsAmt'
        variable_8='CYSlrsCmpEmpBnftPdAmt'
        variable_9='CYTtlFndrsngExpnsAmt'
        variable_10='CYOthrExpnssAmt'
        variable_11='CYTtlPrfFndrsngExpnsAmt'
        variable_12='CYTtlExpnssAmt' 
        #var_1=[ActvtyOrMssnDsc, CYRvnsLssExpnssAmt 上面的变量全部填在这里]
        var_1=[variable_3, variable_4, variable_5, variable_6, variable_7, variable_8, variable_9,
               variable_10, variable_11, variable_12]
        #按照上面var_0里面输入的变量顺序，写下每个变量代表的意思（这将作为最终CSV文件的column名称，一定要按顺序写）：
        #比如： 1, Activity Description 2, Revenues less expenses(current year)
        #1,Total contributions 2,service revenue total 
        #3,Investment income - current year 4,Other revenue - current year 
        #5, Total revenue - current year 6, Grants and similar amounts - current year 
        #7, Benefits paid to members - current year 8, Salaries, etc - current year 
        #9,Total fundraising expense - current year 10, Other expenses - current year 
        #11, Total Expenses - current year

        for i in range(len(var_1)):            
            if var_1[i] in IRS990_result['schedule_parts']['part_i']:

                var_1[i] = IRS990_result['schedule_parts']['part_i'][var_1[i]]

            else:
                var_1[i] = 0
        if 'TtlCntrbtnsAmt' in IRS990_result['schedule_parts']['part_viii']:
            TtlCntrbtnsAmt=IRS990_result['schedule_parts']['part_viii']['TtlCntrbtnsAmt']
        else:
            TtlCntrbtnsAmt = 0
        if 'TtlPrgrmSrvcRvnAmt' in IRS990_result['schedule_parts']['part_viii']:
            TtlPrgrmSrvcRvnAmt =IRS990_result['schedule_parts']['part_viii']['TtlPrgrmSrvcRvnAmt']
        else:
            TtlPrgrmSrvcRvnAmt = 0
        result_990 = [[object_id], [EID], [name], [year], var_0, [TtlCntrbtnsAmt], [TtlPrgrmSrvcRvnAmt]]
        for i in range(len(var_1)):
            result_990.append([var_1[i]])
        return result_990


    def form_990EZ(object_id, name, year, EID):
        
        IRS990EZ_result= xml_runner.run_sked(object_id, 'IRS990EZ').result

        if IRS990EZ_result!=None:
            IRS990EZ_result=IRS990EZ_result[0]
        else:
            re = [[object_id], [EID], [name], [year], ['No Result']]
        #part0没有变量，只有part1的17个
            for i in range(17):
                re.append([0])
            return re

        # 这里写part 0 中需要抓的 variables。 例子： variable_1 = 'FrmtnYr'
        #var_0=[ 填上variables名字, 逗号隔开。例子：variable_name1, variable_name2 ] 
        # var_0 中一共有多少变量？
        #0
        #按照上面var_0里面输入的变量顺序，写下每个变量代表的意思（这将作为最终CSV文件的column名称，一定要按顺序写）： 
        #例子： 1，‘Total Revenue’ 2, 'Total Expense'
        
        if 'OfOrgnztnTrstInd' in IRS990EZ_result['schedule_parts']['ez_part_0']:
            var_0=['Trust']
        elif'OthrOrgnztnDsc' in IRS990EZ_result['schedule_parts']['ez_part_0']:
            var_0=['Other Description']
        elif 'OfOrgnztnCrpInd' in IRS990EZ_result['schedule_parts']['ez_part_0']:
            var_0=['Corporation']
        elif 'OfOrgnztnOthrInd' in IRS990EZ_result['schedule_parts']['ez_part_0']:
            var_0=['Other']
        elif 'OfOrgnztnAsscInd' in IRS990EZ_result['schedule_parts']['ez_part_0']:
            var_0=['Association']
        else:
            var_0=['No Result']
 
        # 这里写part 1 的内容。内容同上面part 0 所要写的一样
        variable_1 ='CntrbtnsGftsGrntsEtcAmt'
        variable_2='PrgrmSrvcRvnAmt'
        variable_3='MmbrshpDsAmt'
        variable_4='InvstmntIncmAmt'
        variable_5c='GnOrLssFrmSlOfAsstsAmt'
        variable_6d='SpclEvntsNtIncmLssAmt'
        variable_7c='GrssPrftLssSlsOfInvntryAmt'
        variable_8='OthrRvnTtlAmt'
        variable_9='TtlRvnAmt'
        variable_10='GrntsAndSmlrAmntsPdAmt'
        variable_11='BnftsPdTOrFrMmbrsAmt'
        variable_12='SlrsOthrCmpEmplBnftAmt'
        variable_13='FsAndOthrPymtTIndCntrctAmt'
        variable_14='OccpncyRntUtltsAndMntAmt'
        variable_15='PrntngPblctnsPstgAmt'
        variable_16='OthrExpnssTtlAmt'
        variable_17='TtlExpnssAmt'

        #var_0=[ 填上variables名字, 逗号隔开。例子：variable_name1, variable_name2 ] 
        var_1= [variable_1 ,variable_2,variable_3,variable_4,variable_5c,variable_6d,variable_7c,variable_8,
                variable_9,variable_10,variable_11,variable_12,variable_13,variable_14,variable_15,variable_16,variable_17]
        # var_0 中一共有多少变量？
        #17
        #按照上面var_0里面输入的变量顺序，写下每个变量代表的意思（这将作为最终CSV文件的column名称，一定要按顺序写）： 
        #例子： 1，‘Total Revenue’ 2, 'Total Expense'
        #1,'Contributions, gifts, grants, and similar amounts received'2,'Program service revenue including government fees and contracts'
        #3,'Membership dues and assessments'4,'Investment income'5,'Gain or (loss) from sale of assets other than inventory'6,'Net income or (loss) from gaming and fundraising events'
        #7,'Gross profit or (loss) from sales of inventory'8,'Other revenue'9,'Total revenue'10,'Grants and similar amounts paid'11,'Benefits paid to or for members'12,'Salaries, other compensation, and employee benefits'
        #13,'Professional fees and other payments to independent contractors'14,'Occupancy, rent, utilities, and maintenance'15,'Printing, publications, postage, and shipping'16,'Other expenses'17,'Total expenses' 
        for i in range(len(var_1)):
            if var_1[i] in IRS990EZ_result['schedule_parts']['ez_part_i']:
                var_1[i] = IRS990EZ_result['schedule_parts']['ez_part_i'][var_1[i]]
            else:
                var_1[i] = 0

        result_990EZ = [[object_id], [EID], [name], [year], var_0]

        for i in range(len(var_1)):
            result_990EZ.append([var_1[i]])
        return result_990EZ

    def form_990PF(object_id, name, year, EID):
        
        IRS990PF_result= xml_runner.run_sked(object_id, 'IRS990PF').result
        if IRS990PF_result!=None:
            IRS990PF_result=IRS990PF_result[0]
        else:
            re = [[object_id], [EID], [name], [year], ['No Result']]
            
            for i in range([26]):
                re.append([0])
            return re
        
        form = 'Private Foundation'
        #写需要的 variables
                #写需要的 variables
        variable1_1a = ['CntrRcvdRvAndExpnssAmt']
        variable_3=['IntrstOnSvRvAndExpnssAmt', ' IntrstOnSvNtInvstIncmAmt', 'IntrstOnSvngsAdjNtIncmAmt']
        variable_4=['DvdndsRvAndExpnssAmt', 'DvdndsNtInvstIncmAmt', 'DvdndsAdjNtIncmAmt']
        variable_5=['GrssRntsRvAndExpnssAmt','GrssRntsNtInvstIncmAmt','GrssRntsAdjNtIncmAmt']
        variable_5b=['a']
        variable_6=['NtGnSlAstRvAndExpnssAmt']
        variable_7=['CpGnNtIncmNtInvstIncmAmt']
        variable_8=['NtSTCptlGnAdjNtIncmAmt']
        variable_9=['IncmMdfctnsAdjNtIncmAmt']
        variable2_10=['GrssPrftRvAndExpnssAmt','GrssPrftAdjNtIncmAmt']
        variable6_11=['OthrIncmRvAndExpnssAmt','OthrIncmNtInvstIncmAmt','OthrIncmAdjNtIncmAmt']
        variable9_12=['TtlRvAndExpnssAmt','TtlNtInvstIncmAmt','TtlAdjNtIncmAmt']
        variable12_13=['CmpOfcrDrTrstRvAndExpnssAmt','CmpOfcrDrTrstNtInvstIncmAmt','CmpOfcrDrTrstAdjNtIncmAmt','CmpOfcrDrTrstDsbrsChrtblAmt']
        variable16_14=['OthEmplSlrsWgsRvAndExpnssAmt','OthEmplSlrsWgsNtInvstIncmAmt','OthEmplSlrsWgsAdjNtIncmAmt','OthEmplSlrsWgsDsbrsChrtblAmt']
        variable20_15= ['PnsnEmplBnftRvAndExpnssAmt','PnsnEmplBnftNtInvstIncmAmt','PnsnEmplBnftAdjNtIncmAmt','PnsnEmplBnftDsbrsChrtblAmt']
        variable24_16a=['LglFsRvAndExpnssAmt','LglFsNtInvstIncmAmt','LglFsAdjNtIncmAmt','LglFsDsbrsChrtblAmt','AccntngFsRvAndExpnssAmt','AccntngFsNtInvstIncmAmt','AccntngFsAdjNtIncmAmt','AccntngFsChrtblPrpsAmt','OthrPrfFsRvAndExpnssAmt','OthrPrfFsNtInvstIncmAmt','OthrPrfFsAdjNtIncmAmt','OthrPrfFsDsbrsChrtblAmt']
        variable36_17=['IntrstRvAndExpnssAmt','IntrstNtInvstIncmAmt','IntrstAdjNtIncmAmt','IntrstDsbrsChrtblAmt']
        variable40_18=['TxsRvAndExpnssAmt','TxsNtInvstIncmAmt','TxsAdjNtIncmAmt','TxsDsbrsChrtblAmt']
        variable44_19=['DprcAndDpltnRvAndExpnssAmt','DprcAndDpltnNtInvstIncmAmt','DprcAndDpltnAdjNtIncmAmt']
        variable48_20=['OccpncyRvAndExpnssAmt','OccpncyNtInvstIncmAmt','OccpncyAdjNtIncmAmt','OccpncyDsbrsChrtblAmt']
        variable52_21=['TrvCnfMtngRvAndExpnssAmt','TrvCnfMtngNtInvstIncmAmt','TrvCnfMtngAdjNtIncmAmt','TrvCnfMtngDsbrsChrtblAmt']
        variable56_22=['PrntngAndPbRvAndExpnssAmt','PrntngAndPbNtInvstIncmAmt','PrntngAndPbAdjNtIncmAmt','PrntngAndPbDsbrsChrtblAmt']
        variable60_23=['OthrExpnssRvAndExpnssAmt','OthrExpnssNtInvstIncmAmt','OthrExpnssAdjNtIncmAmt','OthrExpnssDsbrsChrtblAmt']
        variable64_24=['TtOprExpnssRvAndExpnssAmt','TtOprExpnssNtInvstIncmAmt','TtOprExpnssAdjNtIncmAmt','TtOprExpnssDsbrsChrtblAmt']
        variable68_25=['CntrPdRvAndExpnssAmt','CntrPdDsbrsChrtblAmt']
        variable70_26=['TtlExpnssRvAndExpnssAmt',' TtlExpnssNtInvstIncmAmt','TtlExpnssAdjNtIncmAmt','TtlExpnssDsbrsChrtblAmt']
        #一共多少个变量：
        #73
        #按顺序填入变量：
        var_lst = [variable1_1a,variable_3, variable_4, variable_5, variable_5b, variable_6, variable_7, variable_8, variable_9,variable2_10, variable6_11, variable9_12, variable12_13, variable16_14, variable20_15,
                  variable24_16a, variable36_17, variable40_18, variable44_19, variable48_20, variable52_21,
                  variable56_22, variable60_23, variable64_24,  variable68_25, variable70_26]
        var_1 = []
        for i in range(len(var_lst)):
            var = 0
            for j in range(len(var_lst[i])):
                if var_lst[i][j] in IRS990PF_result['schedule_parts']['pf_part_i']:
                    var= var+eval(IRS990PF_result['schedule_parts']['pf_part_i'][var_lst[i][j]])
                else:
                    var = var+0
            var_1.append(var)
        #按顺序写下每个变量代表的意思：
    #1,Contributions Received - Revenue and Expenses per Books 
    #2,Gross Sales Less Returns and Allowances 3 ,Cost of Goods Sold 4 ,Gross Profit - Revenue and Expenses per Books 5,Gross Profit - Adjusted Net Income 
    #6,Other Income - Revenue and Expenses per Books 7,Other Income - Net Investment Income 8,Other Income - Adjusted Net Income 
    #9,Total - Revenue and Expenses per Books 10,Total - Net Investment Income 11,Total - Adjusted Net Income 
    #12,Compensation of Officers, Directors, Trustees, etc. - Revenue and Expenses per Books 13,Compensation of Officers, Directors, Trustees, etc. - Net Investment Income 14,Compensation of Officers, Directors, Trustees, etc. - Adjusted Net Income 15,Compensation of Officers, Directors, Trustees, etc. - Disbursements for Charitable Purposes 
    #16, Other Employee Salaries and Wages - Revenue and Expenses per Books 17,Other Employee Salaries and Wages - Net Investment Income 18,Other Employee Salaries and Wages - Adjusted Net Income 19,Other Employee Salaries and Wages - Disbursements for Charitable Purposes 
    #20, Pension Plans , Employee Benefits - Revenue and Expenses per Books 21, Pension Plans, Employee Benefits - Net Investment Income 22, Pension Plans, Employee Benefits - Adjusted Net Income 23,Pension Plans, Employee Benefits - Disbursements for Charitable Purposes 
    #24,Legal Fees - Revenue and Expenses per Books 25, Legal Fees - Net Investment Income 26, Legal Fees - Adjusted Net Income 27, Legal Fees - Disbursements for Charitable Purposes 28,Accounting Fees - Revenue and Expenses per Books 29,Accounting Fees - Net Investment Income 30,Accounting Fees - Adjusted Net Income 31,Accounting Fees - Disbursements for Charitable Purposes 32,Other Professional Fees - Revenue and Expenses per Books 33,Other Professional Fees - Net Investment Income 34, Other Professional Fees - Adjusted Net Income 35,Other Professional Fees - Disbursements for Charitable Purposes
    #36, Interest - Revenue and Expenses per Books 37,Interest - Net Investment Income 38,Interest - Adjusted Net Income 39,Interest - Disbursements for Charitable Purposes 
    #40, Taxes - Revenue and Expenses per Books 41, Taxes - Net Investment Income 42,Taxes - Adjusted Net Income 43,Taxes - Disbursements for Charitable Purposes
    #44, Depreciation and Depletion - Revenue and Expenses per Books 45,Depreciation and Depletion - Net Investment Income 46,Depreciation and Depletion - Adjusted Net Income
    #47,Not required to attach Schedule B 
    #48, Occupancy - Revenue and Expenses per Books 49,Occupancy - Net Investment Income 50,Occupancy - Adjusted Net Income 51 ,Occupancy - Disbursements for Charitable Purposes
    #52,Travel, Conferences, and Meetings - Revenue and Expenses per Books 53,Travel, Conferences, and Meetings - Net Investment Income 54,Travel, Conferences, and Meetings - Adjusted Net Income 55, Travel, Conferences, and Meetings - Disbursements for Charitable Purposes
    #56,Printing and Publications - Revenue and Expenses per Books 57, Printing and Publications - Net Investment Income 58,Printing and Publications - Adjusted Net Income 59,Printing and Publications - Disbursements for Charitable Purposes
    #60, Other Expenses - Revenue and Expenses per Books 61, Other Expenses - Net Investment Income 62,Other Expenses - Adjusted Net Income 63,Other Expenses - Disbursements for Charitable Purposes 
    #64,Total Operating and Administrative Expenses - Revenue and Expenses per Books 65,Total Operating and Administrative Expenses - Net Investment Income 66,Total Operating and Administrative Expenses - Adjusted Net Income 67,Total Operating and Administrative Expenses - Disbursements for Charitable Purposes 68,Contributions, Gifts, Grants Paid - Revenue and Expenses per Books 69, Contributions, Gifts, Grants Paid - Disbursements for Charitable Purposes 70, Total Expenses and Disbursements - Revenue and Expenses per Books 71,Total Expenses and Disbursements - Net Investment Income 72,Total Expenses and Disbursements - Adjusted Net Income 73,Total Expenses and Disbursements - Disbursements for Charitable Purposes

        result_990PF = [[object_id], [EID], [name], [year], [form]]
        for i in range(len(var_1)):
            
            result_990PF.append([var_1[i]])
        
        return result_990PF
    try:
        if df_index.loc[index,'RETURN_TYPE'] == '990':
            result_990=form_990(object_id=df_index.loc[index, 'OBJECT_ID'], name=df_index.loc[index, 'TAXPAYER_NAME'], year=df_index.loc[index, 'YEAR'], EID=df_index.loc[index, 'EIN'])
        elif df_index.loc[index,'RETURN_TYPE'] == '990EZ':
            result_990EZ=form_990EZ(object_id=df_index.loc[index, 'OBJECT_ID'], name=df_index.loc[index, 'TAXPAYER_NAME'], year=df_index.loc[index, 'YEAR'], EID=df_index.loc[index, 'EIN'])
        elif df_index.loc[index,'RETURN_TYPE'] == '990PF':
            result_990PF=form_990PF(object_id=df_index.loc[index, 'OBJECT_ID'], name=df_index.loc[index, 'TAXPAYER_NAME'], year=df_index.loc[index, 'YEAR'], EID=df_index.loc[index, 'EIN'])
        if result_990!=[]:
            return [result_990, '990']
        elif result_990EZ != []:
            return [result_990EZ, 'EZ']
        elif result_990PF != []:
            return [result_990PF, 'PF']
    except:
        pass

In [91]:
lst_990=[['Object ID'], ['EID'], ['Name'], ['Year'],['Form'], ['Total contributions'], ['service revenue total'], ['Investment income - current year'], 
         ['Other revenue - current year'], ['Total revenue - current year'],['Grants similar amounts - current year '],['Benefits paid to members - current year'],
         ['Salaries, etc - current year'],['Total fundraising expense'],
         ['Other expenses - current year '],["Total professional fundraising expense"], ['Total Expenses - current year']]
lst_ez=[['Object ID'], ['EID'], ['Name'], ['Year'], ['Form'], ['Contributions'], ['Program service revenue'],
        ['Membership dues and assessments'], ['Investment income'], ['Gain or (loss) from sale of assets other than inventory'], ['Net income or (loss)'],
        ['Gross profit'], ['Other revenue'], ['Total revenue'], ['Grants'], ['Benefits paid'], ['Salaries'], ['Professional fees'],
        ['Occupancy, rent, utilities, and maintenance'],['Printing, publications, postage, and shipping'],['Other expenses'],['Total expenses']]
lst_pf=[['Object ID'], ['EID'], ['Name'], ['Year'],['Form'], 
     ['Total Contributions'],['Interest'], ['Dividends and interest from securities'], ['Gross rents'], ['Net Rental Income'], ['Net Gain from Sale of Assets'], ['Capital Gain Net Income'], ['Net Short-Term Capital Gain'], ['Income Modifications'],['Gross profit'], ['Other revenue'], ['Total revenue'], ['Compensation of officers, directors, trustees, etc'], 
         ['Other employee salaries and wages'], ['Pension plans, employee benefits'], ['Fees'], ['Interest'],
         ['Taxes'], ['Depreciation and depletion'], ['Occupancy'], ['Travel, conferences, and meetings'], ['Printing and publications'],
         ['Other expenses'],['Total operating and administrative expenses'],['Contributions, gifts, grants paid'],
         ['Total expenses and disbursements']]
for index in df_index.index:
    re = func_text_acq(index)

    if re != None and 'EZ' in re:
        lst_ez=[a+b for a,b in zip(lst_ez, re[0])]
    
    elif re != None and '990' in re:
        lst_990=[a+b for a,b in zip(lst_990, re[0])]
        
    elif re != None and 'PF' in re:
        lst_pf=[a+b for a,b in zip(lst_pf, re[0])]
dic_990PF={'Object ID':lst_pf[0], 'EIN':lst_pf[1], 'Name':lst_pf[2], 'Year': lst_pf[3],'Form': lst_pf[4], 'Total Contributions':lst_pf[5],
         'Interest paid':lst_pf[6], 'Dividends and interest from securities':lst_pf[7], 'Gross rents':lst_pf[8], 'Net Rental Income':lst_pf[9], 'Net Gain from Sale of Assets':lst_pf[10], 'Capital Gain Net Income':lst_pf[11], 'Net Short-Term Capital Gain':lst_pf[12], 'Income Modifications':lst_pf[13],
         'Gross profit':lst_pf[14], 'Other revenue':lst_pf[15], 'Total revenue':lst_pf[16],'Compensation of officers or directors etc':lst_pf[17],'Other employee salaries and wages':lst_pf[18], 'Pension plans, employee benefits':lst_pf[19],'Fees':lst_pf[20], 'Interest': lst_pf[21], 'Taxes':lst_pf[22], 'Depreciation and depletion':lst_pf[23], 'Occupancy':lst_pf[24], 'Travel, conferences, and meetings':lst_pf[25], 'Printing and publications':lst_pf[26], 
         'Other expenses':lst_pf[27],'Total operating and administrative expenses':lst_pf[28],'Contributions, gifts, grants paid':lst_pf[29], 'Total expenses and disbursements':lst_pf[30]}
dic_990EZ={'Object ID':lst_ez[0], 'EIN':lst_ez[1], 'Name':lst_ez[2], 'Year': lst_ez[3], 'Form':lst_ez[4],
             'Contributions': lst_ez[5], 'Program service revenue': lst_ez[6], 'Membership dues and assessments':lst_ez[7], 'Investment income': lst_ez[8], 
             'Gain or (loss) from sale of assets other than inventory':lst_ez[9], 'Net income or (loss)':lst_ez[10],
            'Gross profit':lst_ez[11], 'Other revenue':lst_ez[12], 'Total revenue':lst_ez[13], 'Grants':lst_ez[14], 
             'Benefits paid':lst_ez[15], 'Salaries':lst_ez[16], 'Professional fees and other payments':lst_ez[17],
            'Occupancy, rent, utilities, and maintenance':lst_ez[18],'Printing, publications, postage, and shipping':lst_ez[19],'Other expenses':lst_ez[20],'Total expenses':lst_ez[21]}
      
dic_990={'Object ID':lst_990[0], 'EIN':lst_990[1], 'Name':lst_990[2], 'Year': lst_990[3],'Form': lst_990[4], 
     'Total contributions': lst_990[5], 'service revenue total':lst_990[6], 'Investment income - current year': lst_990[7], 
     'Other revenue - current year':lst_990[8], 'Total revenue - current year':lst_990[9],'Grants similar amounts - current year ':lst_990[10],'Benefits paid to members - current year':lst_990[11],
     'Salaries, etc - current year':lst_990[12],'Total fundraising expense - current year':lst_990[13],
     'Other expenses - current year ':lst_990[14],'Total professional fundraising expense - current year':lst_990[15], 'Total Expenses - current year':lst_990[16]}
re_990=pd.DataFrame(data=dic_990)
re_990.to_csv('/home/group03/test990.csv', index = True)
re_990EZ=pd.DataFrame(data=dic_990EZ)
re_990EZ.to_csv('/home/group03/test990ez.csv', index = True)
re_pf=pd.DataFrame(data=dic_990PF)
re_pf.to_csv('/home/group03/test990pf.csv', index = True)
print('done')

Filing version 2012v3.0 isn't supported for this operation
done


In [59]:
df_bmf_2017=pd.read_csv('https://nccs-data.urban.org/data/bmf/2017/bmf.bm1712.csv').query('EIN in @file_EIN')

In [ ]:
df_bmf_2015 = pd.read_csv('https://nccs-data.urban.org/data/bmf/2015/bmf.bm1512.csv').query('EIN in @file_EIN')
df_bmf_2016 =pd.read_csv('https://nccs-data.urban.org/data/bmf/2016/bmf.bm1608.csv').query('EIN in @file_EIN')
df_bmf_2018 =pd.read_csv('https://nccs-data.urban.org/data/bmf/2018/bmf.bm1812.csv').query('EIN in @file_EIN')

In [71]:
df_bmf_2019 = pd.read_csv('https://nccs-data.urban.org/data/bmf/2019/bmf.bm1908.csv').query('EIN in @file_EIN')

In [72]:
df_bmf=pd.concat([df_bmf_2015, df_bmf_2016, df_bmf_2017, df_bmf_2018, df_bmf_2019], ignore_index=True)

In [73]:
df_bmf.groupby('Year').count()

,ACCPER,ADDRESS,ASSETS,CASSETS,CFINSRC,CITY,CTAXPER,CTOTREV,EIN,EPOST,...,cFinSrc,cTaxPer,cTotRev,czFiler,nteeConf,nteeFinal,nteeFinal1,ntmaj10,ntmaj12,ntmaj5
Year,,,,,,,,,,,,,,,,,,,,,
2015,1152,1152,1152,0,0,1152,0,0,1152,1152,...,1127,1127,999,1152,1152,1152,1152,1152,1152,1152
2016,1179,1179,1179,0,0,1179,0,0,1179,1179,...,1157,1157,1020,1179,1179,1179,1179,1179,1179,1179
2017,1150,1154,1153,956,956,1154,956,956,1154,0,...,0,0,0,0,0,0,0,0,0,0
2018,1123,1123,1122,929,929,1123,929,929,1123,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
df_2017=df_index_temp=pd.read_csv('https://s3.amazonaws.com/irs-form-990/index_2017.csv', 
                                  error_bad_lines=False, warn_bad_lines=True # Be cautious with bad lines.
                                 ).query('EIN in @file_EIN')

In [103]:
re=df_2017.merge(df_bmf_2017, on ='EIN', how='outer')
re=re.drop_duplicates()
re.count()

RETURN_ID         659
FILING_TYPE       659
EIN              1277
TAX_PERIOD        659
SUB_DATE          659
TAXPAYER_NAME     659
RETURN_TYPE       659
DLN               659
OBJECT_ID         659
SEC_NAME          174
FRCD             1265
Year             1265
SUBSECCD         1265
TAXPER           1265
ASSETS           1264
INCOME           1264
NAME             1265
ADDRESS          1265
CITY             1265
STATE            1264
NTEECONF            9
NTEEFINAL        1265
NAICS            1262
ZIP5             1265
OUTNCCS          1265
OUTREAS             0
RULEDATE         1265
FIPS             1265
FNDNCD           1265
PMSA              641
MSA_NECH         1089
CASSETS          1050
CFINSRC          1050
CTAXPER          1050
CTOTREV          1050
ACCPER           1261
RANDNUM          1265
NTEECC           1265
NTEE1            1265
LEVEL4           1265
LEVEL1           1265
NTMAJ10          1265
MAJGRPB          1265
LEVEL3           1265
LEVEL2           1265
NTMAJ12   

In [ ]:
df_bmf_2.columns

EIN          1544575
SEC_NAME      418338
FRCD         1544575
SUBSECCD     1544575
TAXPER       1544575
ASSETS       1252079
INCOME       1252079
NAME         1544575
ADDRESS      1544575
CITY         1544575
STATE        1543557
NTEECONF        4627
NTEEFINAL    1544575
NAICS        1540238
ZIP5         1544513
OUTNCCS      1544575
OUTREAS         3112
RULEDATE     1544575
FIPS         1542768
FNDNCD       1544575
PMSA          522598
MSA_NECH     1212852
CASSETS       554722
CFINSRC       554722
CTAXPER       554722
CTOTREV       554722
ACCPER       1074216
RANDNUM      1544575
NTEECC       1544575
NTEE1        1544575
LEVEL4       1544575
LEVEL1       1544575
NTMAJ10      1544575
MAJGRPB      1544575
LEVEL3       1544575
LEVEL2       1544575
NTMAJ12      1544575
NTMAJ5       1544575
FILER        1544575
ZFILER       1544575
dtype: int64

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 11)

SyntaxError: invalid syntax (<ipython-input-10-9016785618fc>, line 35)